In [ ]:
from IPython.display import Javascript, display
import ipywidgets as widgets
from ipywidgets import (
    Button,
    Dropdown,
    GridspecLayout,
    Layout,
    Output,
    RadioButtons,
    Text,
)
from utils import io, visualize

import re

# --- Define smiles input box
smiles_inp = Text(
    placeholder="Enter SMILES...", description="SMILES:", value="[CH]1CO1"
)
smiles_inp.layout = Layout(height="auto", width="auto")


# --- Styling functions
def create_dropdown_list(options):
    return Dropdown(options=options, layout=Layout(height="auto", width="auto"))


def create_expanded_button(description, button_style):
    return Button(
        description=description,
        button_style=button_style,
        layout=Layout(height="auto", width="auto"),
    )


def create_radio_button(description, options):
    return RadioButtons(
        options=options,
        layout={"width": "max-content"},  # If the items' names are long
        description=description,
        disabled=False,
    )


# --- Utility functions
def copy_to_clipboard(value):
    display(Javascript(f"navigator.clipboard.writeText('{value}')"))


# --- Optimization child
opt_layout = GridspecLayout(5, 3, height="700px")
# ----- File options subchild
methods_dropdown = create_dropdown_list(options=["XTB", "REVDSD", "CCSDT"])
xyzs_radio = create_radio_button(
    description="XYZ Files:",
    options=io.get_xyz(smiles=smiles_inp.value, method=methods_dropdown.value),
)
opt_layout[:, 0] = widgets.VBox([methods_dropdown, xyzs_radio])
opt_layout[:4, 1:] = Output()


def update_opt_tab(update):
    xyzs_radio.options = io.get_xyz(
        smiles=smiles_inp.value, method=methods_dropdown.value
    )
    opt_layout[:4, 1:].clear_output()
    xyz_path = xyzs_radio.value
    # --- Visualization subchild
    with opt_layout[:4, 1:3]:
        if xyz_path is not None:
            if "trj" in str(xyz_path) or "allxyz" in str(xyz_path):
                visualize.visualize_traj(xyz_path)
            else:
                visualize.visualize_xyz(xyz_path)
        else:
            try:
                visualize.visualize_smiles(smiles_inp.value)
                opt_layout[4, 1:3] = create_expanded_button("Run optimization", "info")
            except Exception as e:
                print("Enter a valid SMILES string.")
                print(e)
    # --- Copy buttons subchild
    if xyz_path is not None:
        zpv_text, spc_text = io.get_energies(xyz_path)
        zpv_btn = create_expanded_button(zpv_text, "info")
        spc_btn = create_expanded_button(spc_text, "info")

        zpv_value = re.findall(r"[-+]?\d*\.\d+|\d+", zpv_text)
        spc_value = re.findall(r"[-+]?\d*\.\d+|\d+", spc_text)
        zpv_value = zpv_value[0] if zpv_value else ""
        spc_value = spc_value[0] if spc_value else ""

        def on_zpv_click(b):
            copy_to_clipboard(zpv_value)

        def on_spc_click(b):
            copy_to_clipboard(spc_value)

        zpv_btn.on_click(on_zpv_click)
        spc_btn.on_click(on_spc_click)
        opt_layout[4, 1] = zpv_btn
        opt_layout[4, 2] = spc_btn
    else:
        submit_btn = create_expanded_button("Run optimization", "info")

        def on_submit_click(b):
            bash_command = io.optimization(smiles_string=smiles_inp.value)
            copy_to_clipboard(bash_command)

        submit_btn.on_click(on_submit_click)
        opt_layout[4, 1:] = submit_btn


# --- Transition child
trans_layout = GridspecLayout(5, 3, height="700px")
# ----- File options subchild
trans_opts_dropdown = create_dropdown_list(
    options=["Show scan", "Show energy plot", "Show vibrational mode", "New transition"]
)

scans_radio = create_radio_button(
    description="Available transitions:",
    options=io.get_transitions(smiles=smiles_inp.value),
)
allxyzs_dropdown = create_dropdown_list(options=[])
xyzs_radio.options = io.get_xyz(smiles=smiles_inp.value, method=methods_dropdown.value)
trans_layout[:, 0] = widgets.VBox([trans_opts_dropdown, scans_radio])
trans_layout[:4, 1:] = Output()


def update_trans_tab(update):
    # ----- Show scan sublayout
    if trans_opts_dropdown.value == "Show scan":
        allxyzs_dropdown.options = io.get_scan_xyzs(trans_dir=scans_radio.value)
        trans_layout[:, 0] = widgets.VBox(
            [trans_opts_dropdown, scans_radio, allxyzs_dropdown]
        )
        trans_layout[:4, 1:].clear_output()
        xyz_path = allxyzs_dropdown.value
        # --- Visualization subchild
        with trans_layout[:4, 1:]:
            if xyz_path is not None:
                if "trj" in str(xyz_path) or "allxyz" in str(xyz_path):
                    visualize.visualize_traj(xyz_path)
                else:
                    visualize.visualize_xyz(xyz_path)


# --- Define tabs and assign children
tab_contents = ["Optimization", "Transition"]
tab = widgets.Tab()
children = [opt_layout, trans_layout]
tab.children = children
tab.titles = tab_contents

# --- Display
display(smiles_inp, tab)

# --- Observations
methods_dropdown.observe(update_opt_tab, "value")
smiles_inp.observe(update_opt_tab, "value")
xyzs_radio.observe(update_opt_tab, "value")
trans_opts_dropdown.observe(update_trans_tab, "value")
scans_radio.observe(update_trans_tab, "value")
allxyzs_dropdown.observe(update_trans_tab, "value")

Text(value='[CH]1CO1', description='SMILES:', layout=Layout(height='auto', width='auto'), placeholder='Enter S…